## Chapter 6

### Anchors and Counterfactual Explanations
#### Here we will look at Anchors and Counterfactual Explanations as additional post hoc methods

##### We will look at the explanations provided through these methods

This notebook focusses on Anchors Explainability, we will running them on Random Forest Classifier models

We use the iris dataset from sklearn <br>

We are using the sklearn iris datasets. More details on the dataset can be found here <br>
https://scikit-learn.org/1.0/auto_examples/datasets/plot_iris_dataset.html

**Install required libraries**

In [1]:
#https://pypi.org/project/alibi/0.3.1/
!pip install alibi

  Using cached alibi-0.9.6-py3-none-any.whl.metadata (22 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached spacy_lookups_data-1.0.5-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
INFO: pip is looking at multiple versions of spacy[lookups] to determine which version is compatibl

**Import required libraries**

In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from alibi.explainers import AnchorTabular
from sklearn.model_selection import train_test_split

**Load the iris dataset and extract the class types**

In [3]:
dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

**Split the data into train and test set**

In [6]:
X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [4]:
idx = 145
X_train,Y_train = dataset.data[:idx,:], dataset.target[:idx]
X_test, Y_test = dataset.data[idx+1:,:], dataset.target[idx+1:]

**Train the Random Forest Classifier**

In [9]:
np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50)

**Define a lambda function for the prediction probability**

This gets utilised in the Anchor Explainer

In [10]:
predict_fn = lambda x: clf.predict_proba(x)

**Initialize the Anchor Explainer**

In [12]:
explainer = AnchorTabular(predict_fn, feature_names)
explainer.fit(X_train, disc_perc=(25, 50, 75))

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': None, 'disc_perc': (25, 50, 75)},
  'version': '0.9.6'}
)

**Display prediction for first item in test dataset**

In [13]:
idx = 0
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

Prediction:  versicolor


**Display the anchor explainer**

We set the precision threshold to 0.95. This means that predictions on observations where the anchor holds will be the same as the prediction on the explained instance at least 95% of the time.

In [10]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: petal width (cm) > 1.80 AND sepal width (cm) <= 2.80
Precision: 0.98
Coverage: 0.08


The basis which determined the versicolor prediction above is this anchor "petal width (cms) > 1.80 and sepal width (cms) <=2.80.

We used a simple example with fewer features but as the number of dimensions increase, the rules in the anchors provide valuable explanations. 